In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense



In [ ]:
train_path = 'dataset'

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# Save the model
model.save("model/rat_cnn_model.h5")


In [ ]:
# Load the model
model = load_model("model/rat_cnn_model.h5")

test_dir = "testing_folder"

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)
    img = load_img(img_path, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print(f"{img_name} -> No Rat Detected")
    else:
        print(f"{img_name} -> Rat Detected")


In [ ]:
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity

# Extract features from the last dense layer (before final sigmoid)
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

def extract_features(image_path):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    features = feature_extractor.predict(img_array)
    return features


In [ ]:
import random

# Get a few rat image paths from the training dataset
rat_images_path = "dataset/rat"
rat_samples = random.sample(os.listdir(rat_images_path), 5)

# Extract features
test_img_path = os.path.join("testing_folder", os.listdir("testing_folder")[0])
test_features = extract_features(test_img_path)

similarities = []
labels = []

for img_name in rat_samples:
    path = os.path.join(rat_images_path, img_name)
    rat_features = extract_features(path)

    similarity = cosine_similarity(test_features, rat_features)[0][0]
    similarities.append(similarity)
    labels.append(img_name)
